Ce notebook explore les données fictives d'entraînement de trail et calcule le Paramètre Global de Forme (PF). Il examine également les corrélations entre les différentes variables et visualise les résultats.

In [ ]:
# Importation des Bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Configuration des styles de visualisation
sns.set(style="whitegrid")

In [ ]:
# Chargement des Données
df = pd.read_csv('../data/processed/data_with_pf.csv')

# Afficher les premières lignes pour vérifier
df.head()

In [ ]:
# Vérifier les valeurs manquantes
missing_values = df.isnull().sum()
print("Valeurs manquantes par colonne :")
print(missing_values)

In [ ]:
# Sélectionner les colonnes nécessaires
columns = ['Durée (heures)', 'Distance (km)', 'Dénivelé (m)', 
           'FC Moyenne (bpm)', 'VFC (ms)', 'Sommeil (h)', 
           'Qualité Sommeil (1-10)', 'Nutrition (1-10)', 'Hydratation (L)']

data = df[columns].copy()

# Appliquer Min-Max Scaling
scaler = MinMaxScaler()
data_normalized = pd.DataFrame(scaler.fit_transform(data), columns=columns)

# Inverser la normalisation pour FC Moyenne (une FC plus basse est meilleure)
data_normalized['FC Moyenne (bpm)'] = 1 - data_normalized['FC Moyenne (bpm)']

In [ ]:
# Définir les poids
weights = {
    'FC Moyenne (bpm)': 0.20,
    'VFC (ms)': 0.20,
    'Sommeil (h)': 0.20,
    'Qualité Sommeil (1-10)': 0.10,
    'Nutrition (1-10)': 0.10,
    'Hydratation (L)': 0.05,
    'Durée (heures)': 0.05,
    'Distance (km)': 0.05,
    'Dénivelé (m)': 0.05
}

# Calculer le PF
data_normalized['PF'] = data_normalized.apply(lambda row: sum(row[col] * weight for col, weight in weights.items()), axis=1)

# Ajouter le PF au DataFrame original
df['PF Calculé'] = data_normalized['PF']

# Afficher le DataFrame avec le PF Calculé
df.head()

In [ ]:
# Calculer la matrice de corrélation de Pearson
corr_matrix = df[['Durée (heures)', 'Distance (km)', 'Dénivelé (m)', 
                 'FC Moyenne (bpm)', 'VFC (ms)', 'Sommeil (h)', 
                 'Qualité Sommeil (1-10)', 'Nutrition (1-10)', 'Hydratation (L)', 'PF Calculé']].corr(method='pearson')

# Afficher la matrice de corrélation
corr_matrix

In [ ]:
# Heatmap des corrélations
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Heatmap des Corrélations de Pearson')
plt.show()

In [ ]:
# Convertir la colonne Date en datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Tracer le PF au fil du temps
plt.figure(figsize=(10, 5))
plt.plot(df['Date'], df['PF Calculé'], marker='o', linestyle='-', color='b')
plt.title('Paramètre Global de Forme (PF) sur 10 Jours')
plt.xlabel('Date')
plt.ylabel('PF Calculé')
plt.ylim(0, 1)
plt.grid(True)
plt.show()